In [1]:
import pandas as pd
import numpy as np

### Step 1: Information about the NOC version 1.3 update Dec2019
in this step we will uste the information from the official website and the NOC we will use 2 files for this process. in this steps is mandatory to have internet connection in order to download the oficial documents

In [2]:
noc_structure = pd.read_csv("https://www.statcan.gc.ca/eng/statistical-programs/document/noc-cnp-2016-structure-v4-eng.csv")

In [3]:
noc_structure.shape

(690, 5)

In [4]:
noc_structure.sample(10)

,Level,Hierarchical structure,Code,Class title,Class definition
14,4,Unit group,0114,Other administrative services managers,"Other administrative services managers plan, o..."
678,4,Unit group,9537,"Other products assemblers, finishers and inspe...","Other products assemblers, finishers and inspe..."
285,4,Unit group,4033,Educational counsellors,Educational counsellors advise current and pro...
582,4,Unit group,8411,Underground mine service and support workers,Underground mine service and support workers p...
684,4,Unit group,9614,"Labourers in wood, pulp and paper processing","Labourers in wood, pulp and paper processing c..."
270,2,Major group,34,Assisting occupations in support of health ser...,This major group comprises occupations in heal...
637,4,Unit group,9423,Rubber processing machine operators and relate...,Rubber processing machine operators and relate...
306,4,Unit group,4169,Other professional occupations in social scien...,Other professional occupations in social scien...
674,4,Unit group,9533,Other wood products assemblers and inspectors,Other wood products assemblers assemble a vari...
31,4,Unit group,0412,"Government managers - economic analysis, polic...","Government managers in economic analysis, poli..."


In [5]:
noc_elements = pd.read_csv("https://www.statcan.gc.ca/eng/statistical-programs/document/noc-cnp-2016-element-v4-eng.csv")

In [6]:
noc_elements.shape

(43126, 9)

In [7]:
noc_elements.sample(19)

,Level,Code,Class title,Element Type Label English,Element Description English,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
13732,4,3232,Practitioners of natural healing,All examples,herbalist,NaN,NaN,NaN,NaN
41507,4,9537,"Other products assemblers, finishers and inspe...",All examples,pearl earring maker,NaN,NaN,NaN,NaN
32300,4,9211,"Supervisors, mineral and metal processing",All examples,stone products manufacturing foreman/woman,NaN,NaN,NaN,NaN
11185,4,2263,Inspectors in public and environmental health ...,All examples,quarantine officer - public health,NaN,NaN,NaN,NaN
1654,4,411,Government managers - health and social policy...,Main duties,This group performs some or all of the followi...,NaN,NaN,NaN,NaN
34954,4,9416,Metalworking and forging machine operators,All examples,punch press setter - metalworking,NaN,NaN,NaN,NaN
26828,4,7311,Construction millwrights and industrial mechanics,All examples,oil drill rig mechanic,NaN,NaN,NaN,NaN
35653,4,9421,Chemical plant machine operators,All examples,vinyl and asbestos-blending machine tender,NaN,NaN,NaN,NaN
11774,4,2282,User support technicians,Main duties,Participate in the redesign of applications an...,NaN,NaN,NaN,NaN
37445,4,9441,"Textile fibre and yarn, hide and pelt processi...",All examples,winding machine tender - textile manufacturing,NaN,NaN,NaN,NaN


the elements csv file has 9 columns and 43,126 rows and the Stucture file has 5 columns and 690 rows. from the sample exploration, the elements table contains null columns that will be remove before insertion to postgrest


In [8]:
noc_elements = noc_elements.dropna(1)

In [9]:
noc_elements.shape

(43126, 5)

### Step 2: Data Wrangling
In this part we will manage all the data to generated a normalize database in Postgres. 
the main idea is to generate all the necesary Stucture for model Training and Search Engine Support 

In [10]:
noc_structure.columns = ["level","hierarchical_structure","code","title","definition"]

In [11]:
tierOne = noc_structure[noc_structure.level == 1][["code","title","definition"]]
tierOne.columns = ["code_tier_one","title","definition"]

In [12]:
tierTwo = noc_structure[noc_structure.level == 2][["code","title","definition"]]

In [15]:
diff_row_tier = tierTwo[[len(x) > 2 for x in  tierTwo.code]].copy()

In [16]:
data = []
for i in range(len(diff_row_tier)):
    row = diff_row_tier.iloc[i,:].copy()
    start, end = row.code.split("-")
    for n in np.arange(int(start),int(end)+1):
        d = row.copy()
        d.code = str(n).zfill(2)
        data.append(d)

In [17]:
append_data = pd.DataFrame(data)

In [18]:
append_data

,code,title,definition
9,01,Specialized middle management occupations,This major group comprises specialized middle ...
9,02,Specialized middle management occupations,This major group comprises specialized middle ...
9,03,Specialized middle management occupations,This major group comprises specialized middle ...
9,04,Specialized middle management occupations,This major group comprises specialized middle ...
9,05,Specialized middle management occupations,This major group comprises specialized middle ...
56,07,"Middle management occupations in trades, trans...",This major group comprises middle management o...
56,08,"Middle management occupations in trades, trans...",This major group comprises middle management o...
56,09,"Middle management occupations in trades, trans...",This major group comprises middle management o...


In [19]:
tierTwo = tierTwo[~tierTwo.code.isin(code_sep.code)].append(append_data)

In [20]:
tierTwo.sort_values("code",inplace=True)

In [21]:
tierTwo.reset_index(inplace= True,drop=True)

In [22]:
tierTwo.columns = ["code_tier_two","title","definition"]
tierTwo.sample(10)

,code_tier_two,title,definition
34,72,"Industrial, electrical and construction trades",This major group comprises occupations in indu...
14,15,"Distribution, tracking and scheduling co-ordin...","This major group comprises distribution, track..."
16,22,Technical occupations related to natural and a...,This major group comprises technical occupatio...
45,96,"Labourers in processing, manufacturing and uti...",This major group comprises labourers in manufa...
5,05,Specialized middle management occupations,This major group comprises specialized middle ...
44,95,Assemblers in manufacturing,This major group comprises assemblers in manuf...
37,75,Transport and heavy equipment operation and re...,This major group comprises occupations in tran...
22,41,"Professional occupations in law and social, co...",This major group comprises professional occupa...
19,32,Technical occupations in health,This major group comprises technical occupatio...
41,86,"Harvesting, landscaping and natural resources ...",This major group comprises labourers in harves...


In [23]:
tierThree = noc_structure[noc_structure.level == 3][["code","title","definition"]]

In [24]:
tierThree.columns = ["code_tier_three","title","definition"]
tierThree.sample(10)

,code_tier_three,title,definition
49,062,Retail and wholesale trade managers,This minor group includes retail and wholesale...
663,952,"Mechanical, electrical and electronics assemblers",This minor group includes aircraft assemblers ...
486,728,Masonry and plastering trades,"This minor group includes bricklayers, concret..."
74,111,"Auditors, accountants and investment professio...","This minor group includes financial auditors, ..."
392,632,Chefs and cooks,This minor group includes chefs and cooks. The...
491,729,Other construction trades,"This minor group includes roofers, shinglers, ..."
537,745,Longshore workers and material handlers,This minor group includes longshore workers an...
259,322,Technical occupations in dental health care,This minor group includes such technical occup...
280,402,College and other vocational instructors,This minor group includes instructors who teac...
244,314,Therapy and assessment professionals,"This minor group includes audiologists, speech..."


In [25]:
tierFour = noc_structure[noc_structure.level == 4][["code","title","definition"]]

In [26]:
tierFour.columns = ["code_tier_four","title","definition"]
tierFour.sample(20)

,code_tier_four,title,definition
220,2275,Railway traffic controllers and marine traffic...,Railway traffic controllers co-ordinate passen...
196,2232,Mechanical engineering technologists and techn...,Mechanical engineering technologists and techn...
659,9472,"Camera, platemaking and other prepress occupat...",Camera and platemaking workers operate graphic...
285,4033,Educational counsellors,Educational counsellors advise current and pro...
62,0731,Managers in transportation,"Managers in transportation operations plan, or..."
632,9417,Machining tool operators,Machining tool operators set up and operate or...
476,7245,Telecommunications line and cable workers,Telecommunications line and cable workers inst...
672,9531,Boat assemblers and inspectors,"Boat assemblers assemble wooden, fibreglass an..."
666,9523,"Electronics assemblers, fabricators, inspector...",Electronics assemblers and fabricators assembl...
43,0511,"Library, archive, museum and art gallery managers","Library, archive, museum and art gallery manag..."


In [44]:
tierFour

,code_tier_four,title,definition
3,0011,Legislators,Legislators participate in the activities of a...
4,0012,Senior government managers and officials,"Senior government managers and officials plan,..."
5,0013,"Senior managers - financial, communications an...","Senior managers in financial, communications a..."
6,0014,"Senior managers - health, education, social an...","Senior managers in health, education, social a..."
7,0015,"Senior managers - trade, broadcasting and othe...","Senior managers in trade, broadcasting and oth..."
...,...,...,...
685,9615,Labourers in rubber and plastic products manuf...,Labourers in rubber and plastic products manuf...
686,9616,Labourers in textile processing,Labourers in textile processing perform a vari...
687,9617,Labourers in food and beverage processing,Labourers in food and beverage processing perf...
688,9618,Labourers in fish and seafood processing,Labourers in fish and seafood processing perfo...


In [28]:
noc_elements.columns = ["level","code","title","type","description"]

In [29]:
noc_elements["code_tier_four"] = noc_elements.code.apply(lambda t: str(t).zfill(4))
noc_elements["code_tier_three"] = noc_elements.code_tier_four.apply(lambda t4: t4[:3] )
noc_elements["code_tier_two"] = noc_elements.code_tier_four.apply(lambda t4: t4[:2] )
noc_elements["code_tier_one"] = noc_elements.code_tier_four.apply(lambda t4: t4[:1] )

In [31]:
noc_elements[noc_elements.code == 2121]

,level,code,title,type,description,code_tier_four,code_tier_three,code_tier_two,code_tier_one
7685,4,2121,Biologists and related scientists,Illustrative example(s),anatomist,2121,212,21,2
7686,4,2121,Biologists and related scientists,Illustrative example(s),bacteriologist,2121,212,21,2
7687,4,2121,Biologists and related scientists,Illustrative example(s),bioinformatician,2121,212,21,2
7688,4,2121,Biologists and related scientists,Illustrative example(s),biologist,2121,212,21,2
7689,4,2121,Biologists and related scientists,Illustrative example(s),botanist,2121,212,21,2
...,...,...,...,...,...,...,...,...,...
7854,4,2121,Biologists and related scientists,Main duties,Biologists and related scientists may speciali...,2121,212,21,2
7855,4,2121,Biologists and related scientists,Employment requirements,A bachelor's degree in biology or in a related...,2121,212,21,2
7856,4,2121,Biologists and related scientists,Employment requirements,A master's or doctoral degree in biology or a ...,2121,212,21,2
7857,4,2121,Biologists and related scientists,Employment requirements,Post-doctoral research experience is usually r...,2121,212,21,2


In [35]:
information_type = noc_elements["type"].drop_duplicates().to_list()

In [40]:
#for i_type in information_type:
#    noc_elements[noc_elements.type == i_type][["code_tier_four","description"]].sample(5)

In [58]:
type_to_table = {t:"" for t in noc_elements.type.drop_duplicates().to_list()}

In [62]:
type_to_table["Illustrative example(s)"] = "NOC_EXAMPLES"
type_to_table["All examples"] = "NOC_EXAMPLES"
type_to_table["Exclusion(s)"] = "NOC_EXCLUSIONS"
type_to_table["Main duties"] = "NOC_MAIN_DUTIES"
type_to_table["Employment requirements"] = "NOC_EMPLOYMENT_REQUIREMENTS"
type_to_table["Inclusion(s)"] = "NOC_INCLUSIONS"
type_to_table["Additional information"] = "NOC_ADITIONAL_INFORMATION"




In [64]:
type_to_table

{'Illustrative example(s)': 'NOC_EXAMPLES',
 'All examples': 'NOC_EXAMPLES',
 'Exclusion(s)': 'NOC_EXCLUSIONS',
 'Main duties': 'NOC_MAIN_DUTIES',
 'Employment requirements': 'NOC_EMPLOYMENT_REQUIREMENTS',
 'Inclusion(s)': 'NOC_INCLUSIONS',
 'Additional information': 'NOC_ADITIONAL_INFORMATION'}

In [66]:
noc_elements[noc_elements.type == "Main duties"][["code_tier_four","description"]]

,code_tier_four,description
52,0011,This group performs some or all of the followi...
53,0011,"Enact, amend or repeal laws and regulations"
54,0011,Participate in developing or amending governme...
55,0011,"Represent their government at local, national ..."
56,0011,Respond to matters of concern to constituents ...
...,...,...
43118,9619,Check and weigh materials and products
43119,9619,"Sort, pack, crate and package materials and pr..."
43120,9619,"Assist machine operators, assemblers and other..."
43121,9619,Clean work areas and equipment


In [67]:
noc_elements[noc_elements.type == "Additional information"][["code_tier_four","description"]]

,code_tier_four,description
159,0012,There is mobility among senior management occu...
248,0013,There is mobility among senior management occu...
358,0014,There is mobility among senior management occu...
435,0015,There is mobility among senior management occu...
511,0016,There is mobility among senior management occu...
...,...,...
42687,9617,There is some mobility among occupations in th...
42688,9617,Labourers in food and beverage processing may ...
42735,9618,Progression to other occupations in the fish p...
43124,9619,There is some mobility among occupations in th...
